#Parte 1

In [ ]:
!pip install nltk

In [ ]:
!pip install PyPDF2
!pip install fitz
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 30.0 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
import pandas as pd
import PyPDF2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from PyPDF2 import PdfReader


In [ ]:
!python -m spacy download pt_core_news_sm
!python -m spacy download pt


2023-08-08 01:10:51.242406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.5.0/pt_core_news_sm-3.5.0-py3-none-any.whl (13.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
2023-08-08 01:11:04.493061: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 26.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import fitz
import re
import string
import spacy


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/amazon.csv')

df

,reviewText,Positive
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1
...,...,...
19995,this app is fricken stupid.it froze on the kin...,0
19996,Please add me!!!!! I need neighbors! Ginger101...,1
19997,love it! this game. is awesome. wish it had m...,1
19998,I love love love this app on my side of fashio...,1


In [ ]:
def preprocess_text(text):

    # Tokenize the text. deixo tudo lower

    tokens = word_tokenize(text.lower())




    # Remove stop words. na lingua english

    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]




    # Lemmatize the tokens

    lemmatizer = WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]




    # Join the tokens back into a string

    processed_text = ' '.join(lemmatized_tokens)

    return processed_text



In [ ]:
# apply the function df

df['reviewText'] = df['reviewText'].apply(preprocess_text)
df

,reviewText,Positive
0,one best apps acording bunch people agree bomb...,1
1,pretty good version game free . lot different ...,1
2,really cool game . bunch level find golden egg...,1
3,"silly game frustrating , lot fun definitely re...",1
4,terrific game pad . hr fun . grandkids love . ...,1
...,...,...
19995,app fricken stupid.it froze kindle wont allow ...,0
19996,please add ! ! ! ! ! need neighbor ! ginger101...,1
19997,love ! game . awesome . wish free stuff house ...,1
19998,love love love app side fashion story fight wo...,1


In [ ]:
# initialize NLTK sentiment analyzer

analyzer = SentimentIntensityAnalyzer()


# create get_sentiment function

def get_sentiment(text):

    scores = analyzer.polarity_scores(text)

    sentiment = 1 if scores['pos'] > 0 else 0

    return sentiment





In [ ]:

# apply get_sentiment function

df['sentiment'] = df['reviewText'].apply(get_sentiment)

df

,reviewText,Positive,sentiment
0,one best apps acording bunch people agree bomb...,1,1
1,pretty good version game free . lot different ...,1,1
2,really cool game . bunch level find golden egg...,1,1
3,"silly game frustrating , lot fun definitely re...",1,1
4,terrific game pad . hr fun . grandkids love . ...,1,1
...,...,...,...
19995,app fricken stupid.it froze kindle wont allow ...,0,0
19996,please add ! ! ! ! ! need neighbor ! ginger101...,1,1
19997,love ! game . awesome . wish free stuff house ...,1,1
19998,love love love app side fashion story fight wo...,1,1


In [ ]:



print(confusion_matrix(df['Positive'], df['sentiment']))

[[ 1131  3636]
 [  576 14657]]


In [ ]:

print(classification_report(df['Positive'], df['sentiment']))

              precision    recall  f1-score   support

           0       0.66      0.24      0.35      4767
           1       0.80      0.96      0.87     15233

    accuracy                           0.79     20000
   macro avg       0.73      0.60      0.61     20000
weighted avg       0.77      0.79      0.75     20000



In [ ]:


# Lista para armazenar o texto de cada ata
atas = []

# Lista dos caminhos para os arquivos PDF
pdf_files = ["Copom253-not20230322253.pdf", "Copom254-not20230503254.pdf"] # etc...

for file in pdf_files:
    with open(file, 'rb') as f:
        reader = PdfReader(f)
        text = ""
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
        atas.append(text)

# Criar o DataFrame
dfvect = pd.DataFrame({'Atas': atas})

In [ ]:
dfvect["Atas"][0]

'1    bcb.gov.br  \n  \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n \n \n \n \n \n \n \n \nAta da Reunião do  \nComitê de Política Monetária  — Copom  \n21 e 22 de março  de 2023  253ª  253ª Reunião  \nCOPOM  \n21 e 22 de març o \n  \n2    bcb.gov.br   \n \nOs membros do Copom analisaram a evolução recente e as perspectivas para a economia brasileira \ne para a economia internacional, no context o do regime de política monetária, cujo objetivo é \natingir as metas fixadas pelo Conselho Monetário Nacional para a inflação.  \n   \nData:  \n21 e 22  de março de 2023 \n  \nLocal:  Sala s de reuniões do 8º andar (21/3  e 22/3 – manhã) e do 20º andar ( 22/3  – tarde) do \nEdifício -sede do Banco Central do Brasil – Brasília – DF \n  \nHorários de início  \ne término : 21 de març o: 10h05 – 11h34; 14h 10 – 17h24 \n22 de març o: 10h05 – 11h05; 14h 33 – 18h30 \n  \nPresentes :  \nMembros do Copom   Roberto de Oliveira Campos Neto – Presidente  \nBruno Serra 

BR

In [ ]:
# Lista de arquivos PDF
pdf_files = ["Copom253-not20230322253.pdf", "Copom254-not20230503254.pdf","Copom255-not20230621255 (1).pdf","Copom254-not20230503254.pdf",
             "Copom250-not20221026250.pdf","Copom251-not20221207251.pdf"]

# DataFrame para armazenar o texto de todos os PDFs
data = {'Texto': []}

# Iterar através dos arquivos PDF e extrair o texto
for file in pdf_files:
    pdf_documento = fitz.open(file)
    texto_documento = ''
    # Extrair o texto de cada página
    for pagina in pdf_documento:
        texto = pagina.get_text()
        # Limpeza do texto: remover espaços extras, linhas vazias, etc.
        texto_limpo = ' '.join(texto.split())
        texto_documento += texto_limpo
    data['Texto'].append(texto_documento)

# Criar um DataFrame com os textos extraídos

dfvec = pd.DataFrame(data)

# Agora df contém o texto de todos os PDFs na coluna "Texto"
dfvec

,Texto
0,1 bcb.gov.br Ata da Reunião do Comitê de Polít...
1,1 bcb.gov.br Ata da Reunião do Comitê de Polít...
2,1 bcb.gov.br Ata da Reunião do Comitê de Polít...
3,1 bcb.gov.br Ata da Reunião do Comitê de Polít...
4,1 bcb.gov.br Ata da Reunião do Comitê de Polít...
5,1 bcb.gov.br Ata da Reunião do Comitê de Polít...


In [ ]:
pdf_files = ["Copom253-not20230322253.pdf", "Copom254-not20230503254.pdf","Copom255-not20230621255 (1).pdf","Copom254-not20230503254.pdf",
             "Copom250-not20221026250.pdf","Copom251-not20221207251.pdf"]

# DataFrame para armazenar o texto de todos os PDFs
data = {'Texto': []}

# Iterar através dos arquivos PDF e extrair o texto
for file in pdf_files:
    pdf_documento = fitz.open(file)
    texto_documento = ''
    # Extrair o texto de cada página
    for pagina in pdf_documento:
        texto = pagina.get_text()
        texto_documento += texto

    # Dividir o texto pelo primeiro "1." encontrado e pegar tudo depois dele
    partes = texto_documento.split("1.", 1)
    if len(partes) > 1:
        texto_documento = "1." + partes[1]

    # Limpeza do texto: remover espaços extras, linhas vazias, etc.
    texto_limpo = ' '.join(texto_documento.split())
    data['Texto'].append(texto_limpo)

# Criar um DataFrame com os textos extraídos
dfvec = pd.DataFrame(data)

# Agora df contém o texto de todos os PDFs na coluna "Texto" a partir do ponto onde encontra "1."
dfvec

,Texto
0,1. Desde a reunião anterior do Comitê de Polít...
1,1. O ambiente externo se mantém adverso. Os ep...
2,1. O ambiente externo se mantém adverso. Apesa...
3,1. O ambiente externo se mantém adverso. Os ep...
4,1. O ambiente externo mantém-se adverso e volá...
5,1. O ambiente externo mantém-se adverso e volá...


In [ ]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [ ]:
dfvec['Textosinpontos'] = dfvec['Texto'].apply(remove_punctuation)


In [ ]:
nlp = spacy.load('pt_core_news_sm')

# Função para aplicar Lemmatization ao texto
def apply_lemmatization(text):
    return " ".join([token.lemma_ for token in nlp(text)])

# Aplicar a função de Lemmatização à coluna "Texto"
dfvec['Textolem'] = dfvec['Textosinpontos'].apply(apply_lemmatization)


In [ ]:
def preprocess_textbr(text):

    # Tokenize the text. deixo tudo lower

    tokens = word_tokenize(text.lower())




    # Remove stop words. na lingua english

    filtered_tokens = [token for token in tokens if token not in stopwords.words('portuguese')]




    # Lemmatize the tokens

    lemmatizer = WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]




    # Join the tokens back into a string

    processed_text = ' '.join(lemmatized_tokens)

    return processed_text



In [ ]:
# initialize NLTK sentiment analyzer

analyzer = SentimentIntensityAnalyzer()


# create get_sentiment function

def get_sentiment(text):

    scores = analyzer.polarity_scores(text)

    sentiment = 1 if scores['pos'] > 0 else 0

    return sentiment


In [ ]:
dfvec['Textofinlamen'] = dfvec['Textolem'].apply(preprocess_textbr)
dfvec

,Texto,sentimento,Textosinpontos,Textolem,Textofinlamen
0,1. desde reunião anterior comitê política mone...,0,1 desde reunião anterior comitê política monet...,1 desde reunião anterior comitê político monet...,1 desde reunião anterior comitê político monet...
1,1. ambiente externo mantém adverso . episódios...,1,1 ambiente externo mantém adverso episódios e...,1 ambiente externo manter adverso episódio e...,1 ambiente externo manter adverso episódio env...
2,1. ambiente externo mantém adverso . apesar at...,0,1 ambiente externo mantém adverso apesar aten...,1 ambiente externo manter adverso apesar ate...,1 ambiente externo manter adverso apesar atenu...
3,1. ambiente externo mantém adverso . episódios...,1,1 ambiente externo mantém adverso episódios e...,1 ambiente externo manter adverso episódio e...,1 ambiente externo manter adverso episódio env...
4,1. ambiente externo mantém-se adverso volátil ...,0,1 ambiente externo mantémse adverso volátil r...,1 ambiente externo Mantémse adverso volátil ...,1 ambiente externo mantémse adverso volátil re...
5,1. ambiente externo mantém-se adverso volátil ...,0,1 ambiente externo mantémse adverso volátil m...,1 ambiente externo Mantémse adverso volátil ...,1 ambiente externo mantémse adverso volátil ma...


In [ ]:

# apply get_sentiment function

dfvec['sentimento'] = dfvec['Textolem'].apply(get_sentiment)

dfvec

,Texto,sentimento,Textosinpontos,Textolem,Textofinlamen
0,1. desde reunião anterior comitê política mone...,0,1 desde reunião anterior comitê política monet...,1 desde reunião anterior comitê político monet...,1 desde reunião anterior comitê político monet...
1,1. ambiente externo mantém adverso . episódios...,1,1 ambiente externo mantém adverso episódios e...,1 ambiente externo manter adverso episódio e...,1 ambiente externo manter adverso episódio env...
2,1. ambiente externo mantém adverso . apesar at...,0,1 ambiente externo mantém adverso apesar aten...,1 ambiente externo manter adverso apesar ate...,1 ambiente externo manter adverso apesar atenu...
3,1. ambiente externo mantém adverso . episódios...,1,1 ambiente externo mantém adverso episódios e...,1 ambiente externo manter adverso episódio e...,1 ambiente externo manter adverso episódio env...
4,1. ambiente externo mantém-se adverso volátil ...,0,1 ambiente externo mantémse adverso volátil r...,1 ambiente externo Mantémse adverso volátil ...,1 ambiente externo mantémse adverso volátil re...
5,1. ambiente externo mantém-se adverso volátil ...,0,1 ambiente externo mantémse adverso volátil m...,1 ambiente externo Mantémse adverso volátil ...,1 ambiente externo mantémse adverso volátil ma...


#PARTE 2